## Give me the best deck with Champion X

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'./../data/raw/lor_ranked_data_all.csv') # read in the data

In [3]:
champion = 'Viego'
date = '2022-06-20'
n = 50

In [4]:
def best_deck_by_champ(df, champion, date, n):
    df_date = df[df['game_start_time_utc'] > date]
    champ_list = df_date[df_date['archetype'].str.contains(champion)]
    archetypes = champ_list['deck_code'].value_counts() # find the unique archetypes being played
    archetypes = archetypes[archetypes > n] #only keep popular deck_codes
    popular_list = list(archetypes.index) # make a list of popular deck_codes
    champ_list = champ_list.loc[df['deck_code'].isin(popular_list)] # filter df to only keep popular archetypes
    df_champ = champ_list.groupby(['deck_code', 'game_outcome']).size()
    df_champ = df_champ.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
    df_champ.drop('loss', level=1, axis=0, inplace=True) #drop loss %
    df_champ = df_champ.droplevel(level=1) #drop win/loss
    df_champ = df_champ.sort_values(ascending=False)
    
    df_small = df_date[['deck_code', 'archetype']]
    y = pd.DataFrame(df_champ)
    
    inner = pd.merge(y, df_small, on='deck_code', how='inner')#.value_counts()
    inner = inner.value_counts()
    df_champ = inner.sort_values(ascending=False)
    
    return df_champ

In [5]:
best_deck_by_champ(df, champion, date, n) # sorted by playrate

deck_code                                                                                     0     archetype           
CECQCAQDBEAQMAY6AIAQGLRXAIAQKKBRAMCAKNJWG4CACBAFCAAQMAYOAEDAKKYCAECQCNIDAEAQGKQBAECRSAIDAMGQ  52.0  Katarina Elise Viego    3083
CECQCAQDBEAQMAY6AIAQGLRXAIAQKKBRAMCAKNJWG4CACBAFCAAQMAYOAEDAKKYDAECQCGJVAEAQCAZK              52.0  Katarina Elise Viego    2558
CMBQEAIFFAYQGBAFGU3DOCAEA4ATMO2MKFTW26IAAEAQCBIB                                              52.0  Viego (SH)               802
CUCQCAQFBIAQMDABAIDAUGQ7AMCAKAZWG4CQCBI4EAUC4MIBAEAQKAICAEAQKGIBAYCSC                         32.0  Viego Bard               801
CECQCAQDBEAQMAY6AIAQGLRXAMAQKKBRGUBQIBJVGY3QIAIEAUIACBQDBYAQMBJLAIAQKAIZAA                    54.0  Elise Viego (NX)         792
                                                                                                                            ... 
CECAEAIDFY3QEAIFFAYQEBQDBYPAGBAFGU3DOBABAIBQSAIDAUBACBADAIBQCBIBDE2QCAIBAMVA                  36.0  Katar

In [43]:
def best_deck_by_champ_by_winrate():
    x = best_deck_by_champ(df, champion, date, n)
    df_x = pd.DataFrame(x)
    df_x.index.names = ["deck_code", "winrate", "archetype"]
    df_x.reset_index(level=['winrate', 'archetype'], inplace=True)
    df_x = df_x.sort_values(by=['winrate'], ascending=False)
    return df_x
best_deck_by_champ_by_winrate()

winrate  \
deck_code                                                     
CECQCAQDBEAQMAY6AIAQGLRXAIAQKKBRAMCAKNJWG4BQCAY...     67.0   
CECACAIDFYAQMAY6AIAQKKBRAMCAKNJWG4CQCAIDG4AQEAY...     65.0   
CECQCAQDBEBACAZOG4BAMAYODYBQCBIBFAYQGBAFGU3DOAQ...     65.0   
CECACAIDG4AQCBJIAEDAGHQDAQCTKNRXAQAQEAYJAEBQGDI...     65.0   
CMBQGAIFAEUDCAYEAU2TMNYFAQDQCNSMM5WQCAYEA45VC6I...     64.0   
...                                                     ...   
CECQCAQDBEBACAZOG4BACBJIGEBAMAYODYBQIBJVGY3QEAI...     40.0   
CECAEAIDFY3QEAIFFAYQEBQDBYPAGBAFGU3DOBABAIBQSAI...     36.0   
CUCQCAQFBIAQMDABAIDAUGQ7AMCAKAZWG4CQCBI4EAUC4MI...     32.0   
CQCACAIEDQAQIBJXAMAQKAJIGEBQEBIBAIEAIAIDAUGACBI...     29.0   
CECACBQBDQBACBJAGEBQCAIDCYZAGBAFGU3DOAQBAQAQ4BI...     29.0   

                                                               archetype    0  
deck_code                                                                      
CECQCAQDBEAQMAY6AIAQGLRXAIAQKKBRAMCAKNJWG4BQCAY...  Katarina Elise Viego   67  
CECACAIDFYAQMAY6AIAQKKBRAMCAKNJWG4CQCAIDG4AQEAY...  Katarina Elise Viego   68  
CECQCAQDBEBACAZOG4BAMAYODYBQCBIBFAYQGBAFGU3DOAQ...  Katarina Elise Viego   55  
CECACAIDG4AQCBJIAEDAGHQDAQCTKNRXAQAQEAYJAEBQGDI...      Elise Viego (NX)   92  
CMBQGAIFAEUDCAYEAU2TMNYFAQDQCNSMM5WQCAYEA45VC6I...            Viego (SH)   88  
...                                                                  ...  ...  
CECQCAQDBEBACAZOG4BACBJIGEBAMAYODYBQIBJVGY3QEAI...  Katarina Elise Viego   70  
CECAEAIDFY3QEAIFFAYQEBQDBYPAGBAFGU3DOBABAIBQSAI...  Katarina Elise Viego   55  
CUCQCAQFBIAQMDABAIDAUGQ7AMCAKAZWG4CQCBI4EAUC4MI...            Viego Bard  801  
CQCACAIEDQAQIBJXAMAQKAJIGEBQEBIBAIEAIAIDAUGACBI...     Maokai Viego (PZ)   56  
CECACBQBDQBACBJAGEBQCAIDCYZAGBAFGU3DOAQBAQAQ4BI...            Viego (FR)   69  

[81 rows x 3 columns]

In [ ]:
# # double check it worked
# df_date = df[df['game_start_time_utc'] > date] #build dataframe you build in function
# deck = y.index[0][0] # get deck code returned for deck #1 in the function
# true_value = df_date.deck_code.str.count(deck).sum() #see how many times that deck actually appears in dataframe
# output_value = y[0] # get how many times the function said that deck appears
# true_value == output_value # compare values